In [1]:
import h5py
import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv
import os
from os.path import isfile, join
import re
from PIL import Image

In [12]:
##########################################################################################################################
## digits_output, nos hara una lectura de los videos, extraera los datos de las horas y los almacenara en un archivo .csv#
##########################################################################################################################

def digits_output (directorio):
    
    #Para contar los frames creo una funcion para hacer un contador 
    fotograma = 0
    
    Not_found = 0
    
    video = cv2.VideoCapture(directorio)
    
    archivo_csv = directorio[11:]
    
    #elimino las comas de los nombres para evitar futuros conflictos
    removed = archivo_csv.replace(",", "")
   
    #el titulo de nuestra tabla
    header = ['Frame','Hours', 'Minutes', 'Seconds']
    
    #escribimos la cabezera en nuestro archivo .csv
    with open('RAT_VIDEOS_CSV/'+ str(removed) + '.csv', 'wt') as f:
        csv_writer = csv.writer(f)
        csv_writer.writerow(header) # write header


        
    #creamos la biblioteca donde almacenaremos el estado de los segmentos para cada digito
    DIGITS_LOOKUP = {(1, 1, 1, 0, 1, 1, 1): 0, (0, 0, 1, 0, 0, 1, 0): 1,
                     (1, 0, 1, 1, 1, 0, 1): 2, (1, 0, 1, 1, 0, 1, 1): 3,
                     (0, 1, 1, 1, 0, 1, 0): 4, (1, 1, 0, 1, 0, 1, 1): 5,
                     (1, 1, 0, 1, 1, 1, 1): 6, (1, 0, 1, 0, 0, 1, 0): 7,
                     (1, 1, 1, 1, 1, 1, 1): 8, (1, 1, 1, 1, 0, 1, 1): 9}

    #loop que se encargara de leer el video 
    while (video.isOpened()):
        ret, frame = video.read()
    
        if ret == True:

            #Creo un array donde voy a meter los digitos que voy a recortar
            digits_frame = []
        
            #Esta lista almacenara los digitos que se hayan reconocido
            digits = []
            
            #arrays en los que almacenare los frames, las horas, los minutos y los segundos
            array_frames = []
            
            array_hours = []
            array_minutes = []
            array_seconds = []
        
            #recortamos cada la imagen para tener solo la hora
            hora = frame[4:38, 290:485]
            
            #Para evitar encontrar errores, pasaremos un filtro que detecte solo los blancos. 
            #Establecemos el rango mínimo y máximo de (Blue, Green, Red):
            white_low = np.array([175,175,175])
            white_hight = np.array([255, 255, 255])
            # Recordatorio: el rango sirve para determinar qué píxeles detectaremos.
            # Cada píxel tiene asignado un valor [B, G, R] según su cantidad de Azul, Verde y Rojo.
            # Con el rango que hemos definido, detectaremos los píxeles que cumplan estas tres condiciones:

            #Detectamos los píxeles que estén dentro del rango que hemos establecido:
            mask = cv2.inRange(hora, white_low, white_hight)
        
            #con threshold pasamos la foto a binario
            thresh = cv2.threshold(mask, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
        
            #recortamos los digitos de forma manual de la imagen binerizada y añadimos estos digitos al array digitos
            digit_1 = thresh[0:55,2:25]
            digits_frame.append(digit_1)
            digit_2 = thresh[0:55,25:48]
            digits_frame.append(digit_2)
        
            digit_3 = thresh[0:55,75:98]
            digits_frame.append(digit_3)
            digit_4 = thresh[0:55,98:121]
            digits_frame.append(digit_4)

            digit_5 = thresh[0:55,145:168]
            digits_frame.append(digit_5)
            digit_6 = thresh[0:55,170:193]
            digits_frame.append(digit_6)
            
            fotograma +=1 
        
            #creo un loop que recorra el array de los digitos, en este loop hago los recortes manuales de los segmentos y los almaceno
            #en otro array vacio
            for c in digits_frame:
                #en este array almacenaremos los segmentos 
                segmentos = []
            
                #recorto uno a uno los segmentos del digito y los añado al array
                segmento_0 = c[0:5,:]
                segmentos.append(segmento_0)
                segmento_1 = c[5:15,1:7]
                segmentos.append(segmento_1)
                segmento_2 = c[5:15, 17:23]
                segmentos.append(segmento_2)
                segmento_3 = c[14:19,:]
                segmentos.append(segmento_3)
                segmento_4 = c[19:29,1:7]
                segmentos.append(segmento_4)
                segmento_5 = c[19:29:,17:23]
                segmentos.append(segmento_5)
                segmento_6 = c[29:34,:]
                segmentos.append(segmento_6)
            
                #creo una lista de 0, con longitud segmentos (7)
                on = [0]* len(segmentos)
            
                #este loop va a server para recorrer ahora el array de los segmentos
                for i in range(len(segmentos)):
                    segmento=segmentos[i]
                    total = cv2.countNonZero(segmento) #countNonZero, cuenta el numero de no ceros, es decir el numero de pixeles que no son 0
                    height, width= segmento.shape
                    area = height * width #calculo el area total de la imagen
                
                    if total / float(area) > 0.3: #esto lo que hace es calcular la densidad de pixeles que no son 0, si esta densidad
                        #es mayor al 30% diremos que el segmento esta encendido y por lo tanto añadiremos un 1
                        #a la lista on, en la posicion correspondiente
                        on[i]= 1
                
                
                #debido a que habra numeros que no se encuentren en nuestro diccionario, hare un try and except
                #intentara buscar los numeros en el diccionario y si no los encuentra no hara nada.
                try:
                    digit = DIGITS_LOOKUP[tuple(on)] #busca nuestra lista on en el diccionario DIGITS_LOOKUP, si lo encuentra lo añade
                                                #a la lista digit que sera la cual almacene todos los digitos reconocidos
                    digits.append(digit)
                                
                    
                except:
                    digits = None             #Si no lo encuentra saldra como None
                    
                
        
                
            #en este if lo que hacemos es separar los numeros reconocidos en su unidad de tiempo correspondiente
            if digits is not None:
                hours = str(digits[0])+str(digits[1])
                minutes = str(digits[2])+str(digits[3])
                seconds = str(digits[4])+str(digits[5])
                
                #la siguiente funcion lo unico que hace es colocar los numeros del array digits en formato hora 00:00:00
                #formato = str(digits[0])+str(digits[1])+(':')+ str(digits[2])+str(digits[3])+(':')+str(digits[4])+str(digits[5])
                
            else:
                hours = 'None'
                minutes = 'None'
                seconds = 'None'
                
                #formato = 'None'
                Not_found += 1
            
            #añadimos en sus arrays correspondientes las horas, los minutos y los segundos 
            array_hours.append(hours)
            array_minutes.append(minutes)
            array_seconds.append(seconds)
            
            #array_hora.append(formato)
            
            #añadimos en el array_frames, el numero de frames en formato cuenta de 1 a X
            array_frames.append(fotograma)
            
            #print (fotograma, array_hora)
            
            
            #cada vez que hagamos este proceso, añadiremos al archivo .csv los arrays de las horas y frames
            with open('RAT_VIDEOS_CSV/'+ str(removed) + '.csv', 'a') as f:
                csv_writer = csv.writer(f)
                csv_writer.writerow(array_frames + array_hours + array_minutes + array_seconds)

    
        if ret == False:
            break
        
    print (removed, 'No encontrados: ',Not_found)

In [13]:
##########################################################################################################################
# Creamos una lista con todos los archivos que tenemos en la carpeta de los videos, y ejecutamos la funcion anterior a cada
# uno de ellos
##########################################################################################################################

#Aqui vamos a ejecutar el programa para todos los archivos de una carpeta

#Le decimos la ruta 
ruta = r'C:\Users\flum1\Desktop\Deep Learning\RAT_VIDEOS'

#hacemos una lista de todos los archivos de la ruta
archivo = os.listdir(ruta)

#un loop para que ejecute el programa anterior a todos los elementos de la lista
for i in archivo:
       digits_output('RAT_VIDEOS/' + i)

1.2018-1-9rat8y9_20180110_131607.mp4 No encontrados:  1
10.2017-11-10rat11y12_20171110_172546.mp4 No encontrados:  16
100.2018-2-6rata34y36_20180207_081604.mp4 No encontrados:  4
101.2018-2-6rata34y36_20180207_101607.mp4 No encontrados:  1
102.2018-2-7rata34y36_20180207_163054.mp4 No encontrados:  9
103.2018-2-7rata34y36_20180207_174056.mp4 No encontrados:  1
104.2018-2-7rata34y36_20180207_182057.mp4 No encontrados:  4
105.2018-2-20rata34y36_20180221_083038.mp4 No encontrados:  6
106.2018-2-20rata34y36_20180221_101041.mp4 No encontrados:  1
107.2018-2-20rata34y36_20180221_115043.mp4 No encontrados:  2
108.2018-2-20rata34y36_20180221_130046.mp4 No encontrados:  1
109.2018-2-20rata34y36_20180221_142048.mp4 No encontrados:  4
11.2017-11-13rat11y12_20171112_182001.mp4 No encontrados:  7
110.2018-2-20rata34y36_20180221_153050.mp4 No encontrados:  2
111.2018-2-21rata34y36_20180221_184011.mp4 No encontrados:  0
112.2018-4-2rata31y34_20180402_164606.mp4 No encontrados:  6
113.2018-4-2rata31y34

45.2017-12-25rata2425y28_20171226_084721.mp4 No encontrados:  2
46.2017-12-25rata2425y28_20171226_105725.mp4 No encontrados:  9
47.2017-12-25rata2425y28_20171226_002706.mp4 No encontrados:  27
48.2017-12-26rata25y28_20171226_142925.mp4 No encontrados:  2
49.2017-12-26rata25y28_20171226_162928.mp4 No encontrados:  10
5.2017-11-10rat11y12_20171110_075155.mp4 No encontrados:  15
50.2017-12-22rata2627y30_20171223_053350.mp4 No encontrados:  1
51.2017-12-26rata2627y30_20171226_173948.mp4 No encontrados:  27
52.2017-12-26rata2627y30_20171226_195952.mp4 No encontrados:  7
53.2017-12-26rata2627y30_20171226_223956.mp4 No encontrados:  5
54.2018-1-6rata2627y30_20180107_154717.mp4 No encontrados:  3
55.2018-1-6rata2627y30_20180107_170720.mp4 No encontrados:  10
56.2018-1-7rata2627y30_20180107_192111.mp4 No encontrados:  14
57.2018-1-7rata2627y30_20180107_193111.mp4 No encontrados:  1
58.2018-1-7rata2627y30_20180107_202113.mp4 No encontrados:  1
59.2018-1-7rata2627y30_20180107_212115.mp4 No encont

In [21]:
##########################################################################################################################
# Al tener datos clasificados como None, tendremos que rellenar todos esos datos no renocidos por nuestro programa,
# para ello hacemos una extrapolación. Con esto obtenemos una predicción de los datos desconocidos.
# Una vez obtenidos todos los datos al 100%, volveremos a crear todos los archivos .csv de nuevo, pero esta vez haremos
# un archivo para cada rata
##########################################################################################################################
def digits_output_completed (ruta):
    
    #hacemos una lista de todos los archivos de la ruta, vamos a recorrer todos los archivos .csv para corregirlos
    archivo = os.listdir(ruta)
    
    #un loop para que ejecute el programa anterior a todos los elementos de la lista
    for name in archivo:
        #Como hay frames no encontrados, hacemos esto para extrapolar los datos y rellenar los 'Nones'
        
        #Numero de frame cuando es None
        x = []
        
        #Numero de frame cuando NO es None
        xp = []

        #la hora, en segundos
        fp = []


        #Leemos el archivo csv obtenido anteriormente para extraer los datos
        with open('RAT_VIDEOS_CSV/'+ name, newline='') as File:  
            reader = csv.reader(File)
      
            #Hacemos un loop que recorra todos los elementos de nuestro archivo
            for line in reader:
                
                #Como tenemos espacios en blanco, solo haremos que funcione si la longitud es de 4 elementos
                if len(line) == 4:
                    
                    if line[0] != 'Frame' and line[1] != 'None':
                        xp.append(int(line[0]))
            
                    if  line[0] != 'Frame' and line[1] == 'None':
                        x.append(int(line[0]))
                
                    if line[1] != 'Hours' and line[2] != 'Minutes' and line[3] != 'Seconds' and line[1] != 'None':
                
                        #sera necesario pasar ls horas minutos y segundos a una sola unidad (segundos)
                        h_m = int(line[1]) * 3600
                        m_s = int(line[2]) * 60
                        s = int(line[3])
                        total = h_m + m_s + s
                
                        fp.append(total)
                
                
        #para averiguar los valores desconocidos vamos a extrapolar, con esto hacemos una predicción de los valores siguientes 
        #a partir de los valores ya conocidos
        extrapola = np.polyfit(xp, fp, 1)

        #pasamos la lista x a array para poder hacer las operaciones
        x_array = np.array(x)

        #el valor de y sera los segundos correspondientes a las x cuando son 'None'
        y = extrapola[0]*x_array + extrapola[1]

        #redondeamos el valor obtenido
        y_round = np.round(y)

        #Creamos una cuenta que va a ser el nuevo indice
        index_replacement = 0

        #Creamos los arrays donde vamos a almacenar los datos antiguos y los que hemos obtenido en la extrapolacion
        frame_num  = []
        hora_stamp = []
        min_stamp  = []
        sec_stamp  = []


        #Volvemos a hacer una lectura del archivo, esta vez para asignar los valores que eran desconocidos a su lugar correspondiente
        with open('RAT_VIDEOS_CSV/'+ name , newline='') as File:
            reader = csv.reader(File)
            
            for line in reader:
                
                if len(line) == 4:
                    
                    if line[0] == 'Frame':
                        continue
                    
                    #almacenamos en el array los frames
                    frame_num.append(line[0])
                    
                    if line[1] == 'None':
                        extrapolated_time_stamp = y_round[ index_replacement ]
                        
                        hor=int(extrapolated_time_stamp/3600)
                        minu=int((extrapolated_time_stamp-(hor*3600))/60)
                        seg=int(extrapolated_time_stamp-((hor*3600)+(minu*60)))
            
            
                        hora_stamp.append(str(hor))
                        min_stamp.append (str(minu))
                        sec_stamp.append (str(seg))
            
                        index_replacement += 1
                
                    else:
                        hora_stamp.append(line[1])
                        min_stamp.append (line[2])
                        sec_stamp.append (line[3])



        # Ahora que tenemos las arrays frame_num, hora_stamp, min_stamp y sec_stamp, podemos crear los archivos .csv 
        #(uno por rata) y guardarlos.

        #el titulo de nuestra tabla
        header = ['Frame','Hours', 'Minutes', 'Seconds']
    
        #Obtengo una subcadena para obtener solo la parte donde aparecen los numeros de las ratas
        a = name.find ('rat')
        b = name.find ('_')
        
        cadena = re.sub("\D", "", name[a:b]) #me devolvera los numeros encontrados entre las posiciones a:b
        cadena = int(cadena)
        
        
        #En los videos siempre disponemos de minimo 2 ratas, por lo que para el reconocimiento de numeros opto por las
        #siguientes dos posibilidades
        
        #Si la cadena es un numero inferior a 100, querra decir que disponemos de dos unidades que tendremos que separar
        if cadena < 100:
            string1 = str(cadena)
            
            for n in string1:
                
                with open('RAT_VIDEOS_COMPLETED/'+ name[:a]+'.Rat'+ n +'.csv', 'wt') as f:
                    writer = csv.writer(f)
                    writer.writerow(header)
                    
                    for t in zip(frame_num, hora_stamp, min_stamp, sec_stamp):
                        writer.writerow(t)
                        
        #Si la cadena es un numero mayor a 100, querra decir que disponemos de dos o mas numeros, comprobando su longitud
        #averiguaremos de cuantos y que numeros son
        if cadena > 100:

            #array donde almacenamos los numeros
            dec2 = []
            string2 = str(cadena)


            #Longitud 4 = 2 numeros = 2 archivos
            if len(string2) == 4:
                dec2.append(string2[:2])
                dec2.append(string2[2:4])
                
                if len(dec2) == 2:
                    
                    for n in dec2:
                        
                        with open('RAT_VIDEOS_COMPLETED/'+ name[:a]+'.Rat'+ n +'.csv', 'wt') as f:
                            writer = csv.writer(f)
                            writer.writerow(header)

                            for t in zip(frame_num, hora_stamp, min_stamp, sec_stamp):
                                writer.writerow(t)
                                
            #Longitud 6 = 3 numeros = 3 archivos
            if len(string2) == 6:
                dec2.append(string2[:2])
                dec2.append(string2[2:4])
                dec2.append(string2[4:6])
                
                if len(dec2) == 3:
                    
                    for n in dec2:
                        
                        with open('RAT_VIDEOS_COMPLETED/'+ name[:a]+'.Rat'+ n +'.csv', 'wt') as f:
                            writer = csv.writer(f)
                            writer.writerow(header)

                            for t in zip(frame_num, hora_stamp, min_stamp, sec_stamp):
                                writer.writerow(t)
                                
            #Longitud 8 = 4 numeros = 4 archivos
            if len(string2) == 8:
                dec2.append(string2[:2])
                dec2.append(string2[2:4])
                dec2.append(string2[4:6])
                dec2.append(string2[6:8])


                if len(dec2) == 4:
                    
                    for n in dec2:
                        
                        with open('RAT_VIDEOS_COMPLETED/'+ name[:a]+'.Rat'+ n +'.csv', 'wt') as f:
                            writer = csv.writer(f)
                            writer.writerow(header)

                            for t in zip(frame_num, hora_stamp, min_stamp, sec_stamp):
                                writer.writerow(t)
                                
            #Longitud 10 = 5 numeros = 5 archivos
            if len(string2) == 10:
                dec2.append(string2[:2])
                dec2.append(string2[2:4])
                dec2.append(string2[4:6])
                dec2.append(string2[6:8])
                dec2.append(string2[8:10])


                if len(dec2) == 5:
                    
                    for n in dec2:
                    
                        with open('RAT_VIDEOS_COMPLETED/'+ name[:a]+'.Rat'+ n +'.csv', 'wt') as f:
                            writer = csv.writer(f)
                            writer.writerow(header)

                            for t in zip(frame_num, hora_stamp, min_stamp, sec_stamp):
                                writer.writerow(t)
                                
        print(name + ' ' + 'completed')

In [22]:
#Ejecutamos la funcion anterior
digits_output_completed(r'C:\Users\flum1\Desktop\Deep Learning\RAT_VIDEOS_CSV')

1.2018-1-9rat8y9_20180110_131607.mp4.csv completed
10.2017-11-10rat11y12_20171110_172546.mp4.csv completed
100.2018-2-6rata34y36_20180207_081604.mp4.csv completed
101.2018-2-6rata34y36_20180207_101607.mp4.csv completed
102.2018-2-7rata34y36_20180207_163054.mp4.csv completed
103.2018-2-7rata34y36_20180207_174056.mp4.csv completed
104.2018-2-7rata34y36_20180207_182057.mp4.csv completed
105.2018-2-20rata34y36_20180221_083038.mp4.csv completed
106.2018-2-20rata34y36_20180221_101041.mp4.csv completed
107.2018-2-20rata34y36_20180221_115043.mp4.csv completed
108.2018-2-20rata34y36_20180221_130046.mp4.csv completed
109.2018-2-20rata34y36_20180221_142048.mp4.csv completed
11.2017-11-13rat11y12_20171112_182001.mp4.csv completed
110.2018-2-20rata34y36_20180221_153050.mp4.csv completed
111.2018-2-21rata34y36_20180221_184011.mp4.csv completed
112.2018-4-2rata31y34_20180402_164606.mp4.csv completed
113.2018-4-2rata31y34_20180402_164606.mp4.csv completed
114.2018-4-2rata31y34_20180402_173607.mp4.csv 

58.2018-1-7rata2627y30_20180107_202113.mp4.csv completed
59.2018-1-7rata2627y30_20180107_212115.mp4.csv completed
6.2017-11-10rat11y12_20171110_083156.mp4.csv completed
60.2018-1-7rata2627y30_20180107_213115.mp4.csv completed
61.2018-1-7rata2627y30_20180107_234119.mp4.csv completed
62.2018-1-7rata2627y30_20180107_235119.mp4.csv completed
63.2018-2-13rata2526y30_20180213_191104.mp4.csv completed
64.2018-2-13rata2526y30_20180213_194105.mp4.csv completed
65.2018-2-13rata2526y30_20180213_203106.mp4.csv completed
66.2018-2-13rata2526y30_20180213_212107.mp4.csv completed
67.2018-2-13rata2526y30_20180214_001112.mp4.csv completed
68.2018-2-13rata2526y30_20180214_005114.mp4.csv completed
69.2018-2-13rata2526y30_20180214_020116.mp4.csv completed
7.2017-11-10rat11y12_20171110_091157.mp4.csv completed
70.2018-2-13rata2526y30_20180214_032118.mp4.csv completed
71.2018-2-13rata2526y30_20180214_045121.mp4.csv completed
72.2018-2-13rata2526y30_20180214_062123.mp4.csv completed
73.2018-2-13rata2526y30_2

In [23]:
##########################################################################################################################
# Como en cada video salen varias ratas, ademas de crear un archivo para cada rata, tambien tendremos que tener un video
# para cada una. La siguiente funcion recorta los videos en tantos trozos como numeros tenga en su nombre
##########################################################################################################################
def cropped_video (directorio):
    
    video = cv2.VideoCapture(directorio)

    name = directorio[11:]

    #El reconomcimiento de numeros en el enunciado sera el mismo que en la funcion digits_output_completed
    a = name.find ('rat')
    b = name.find ('_')
    cadena = re.sub("\D", "", name[a:b])
    cadena = int(cadena)

    
    if cadena < 100:
        
        #array donde almacenamos los numeros
        dec1 = []
        string1 = str (cadena)
        
        #Recorremos la cadena
        for n in string1:
            dec1.append(n)


        #al reconocer 2 numeros, sabremos que tenemos 2 ratas. Por lo tanto haremos dos recortes en el video.
        if len(dec1) == 2:

            cap = cv2.VideoCapture('RAT_VIDEOS/'+ name )
            
            #Creamos dos videos distintos, uno para cada rata
            corte1 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec1[0]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(620,620))
            corte2 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec1[1]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(620,620))

            while (cap.isOpened()):
                ret, frame = cap.read()

                if ret == True:
                    
                    #recortes de video
                    izq = frame[100:, :620]
                    dch = frame[100:, 640:1260]
                    
                    corte1.write(izq)
                    corte2.write(dch)


                if ret == False:
                    break


            corte1.release()
            corte2.release()
    
    #Si la cadena es superior a 100, se tratara de numeros de dos unidades, por lo que los separaremos de dos en dos para
    #obtener uno solo
    if cadena > 100:
        
        #array donde almacenamos los numeros
        dec2 = []
        string2 = str(cadena)
        
        
        #Longitud 4 = 2 numeros = 2 videos
        if len(string2) == 4:
            dec2.append(string2[:2])
            dec2.append(string2[2:4])

            
            if len(dec2) == 2:

                cap = cv2.VideoCapture('RAT_VIDEOS/'+ name )
                corte1 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[0]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(620,620))
                corte2 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[1]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(620,620))

                while (cap.isOpened()):
                    ret, frame = cap.read()

                    if ret == True:
                        izq = frame[100:, :620]
                        corte1.write(izq)

                        dch = frame[100:, 640:1260]
                        corte2.write(dch)


                    if ret == False:
                        break


                corte1.release()
                corte2.release()
        
        #Longitud 6 = 3 numeros = 3 videos
        if len(string2) == 6:
            dec2.append(string2[:2])
            dec2.append(string2[2:4])
            dec2.append(string2[4:6])


            if len(dec2) == 3:

                cap = cv2.VideoCapture('RAT_VIDEOS/'+ name )
                corte1 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[0]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(500,500))
                corte2 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[1]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(600,420))
                corte3 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[2]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(620,500))

                while (cap.isOpened()):
                    ret, frame = cap.read()

                    if ret == True:
                        izq = frame[220:, :500]
                        corte1.write(izq)

                        dch = frame[300:, 680:]
                        corte2.write(dch)

                        cen = frame[:500, 300:920]
                        corte3.write(cen)


                    if ret == False:
                        break


                corte1.release()
                corte2.release()
                corte3.release()

        #Longitud 8 = 4 numeros = 4 videos
        if len(string2) == 8:
            dec2.append(string2[:2])
            dec2.append(string2[2:4])
            dec2.append(string2[4:6])
            dec2.append(string2[6:8])


            if len(dec2) == 4:

                cap = cv2.VideoCapture('RAT_VIDEOS/'+ name )
                corte1 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[0]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(620,420))
                corte2 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[1]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(660,420))
                corte3 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[2]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(680,420))
                corte4 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[3]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(620,420))

                while (cap.isOpened()):
                    ret, frame = cap.read()

                    if ret == True:

                        izq = frame[300:, :620]
                        corte1.write(izq)

                        dch = frame[300:, 620:]
                        corte2.write(dch)

                        izq2 = frame[:420, :680]
                        corte3.write(izq2)

                        dch2 = frame[:420, 660:]
                        corte4.write(dch2)


                    if ret == False:
                        break


                corte1.release()
                corte2.release()
                corte3.release()
                corte4.release()

                
        #Longitud 10 = 5 numeros = 5 videos
        if len(string2) == 10:
            dec2.append(string2[:2])
            dec2.append(string2[2:4])
            dec2.append(string2[4:6])
            dec2.append(string2[6:8])
            dec2.append(string2[8:10])


            if len(dec2) == 5:

                cap = cv2.VideoCapture('RAT_VIDEOS/'+ name )
                corte1 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[0]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(600,370))
                corte2 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[1]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(680,370))
                corte3 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[2]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(500,350))
                corte4 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[3]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(430,350))
                corte5 = cv2.VideoWriter('RAT_VIDEOS_CROPPED/'+name[:a]+'.Rat'+ dec2[4]+'.mp4', cv2.VideoWriter_fourcc(*'XVID'),20.0,(400,350))

                while (cap.isOpened()):
                    ret, frame = cap.read()

                    if ret == True:

                        izq = frame[350:, :600]
                        corte1.write(izq)

                        dch = frame[350:, 600:]
                        corte2.write(dch)

                        cen = frame[:350, 350:850]
                        corte3.write(cen)

                        dch2 = frame[:350, 850:]
                        corte4.write(dch2)

                        izq2 = frame[:350, :400]
                        corte5.write(izq2)


                    if ret == False:
                        break


                corte1.release()
                corte2.release()
                corte3.release()
                corte4.release()
                corte5.release()

In [ ]:
##########################################################################################################################
# Creamos una lista con todos los archivos que tenemos en la carpeta de los videos, y ejecutamos la funcion anterior a cada
# uno de ellos
##########################################################################################################################

#Le decimos la ruta 
ruta = r'C:\Users\flum1\Desktop\Deep Learning\RAT_VIDEOS'

#hacemos una lista de todos los archivos de la ruta
archivo = os.listdir(ruta)

#un loop para que ejecute el programa anterior a todos los elementos de la lista
for i in archivo:
       cropped_video('RAT_VIDEOS/' + i)

In [27]:
##########################################################################################################################
# El siguiente paso sera etiquetar todos los videos, es decir, en cada frame de video etiquetar en que estado se encuentra
#la rata. Dos posibilidades, Ataque y reposo
##########################################################################################################################

video_name = [] #array donde almacenaremos el nombre del video
rat_number = [] #array donde almacenaremos el numero de la rata que sufre el ataque
time_start = [] #array donde almacenaremos el tiempo en el que empieza el ataque
time_end = [] #array donde almacenaremos el tiempo en el que finaliza el ataque


#los tiempos de los ataques han sido proporcionados por el laboratorio en un archivo excel, el cual tendremos que cambiar
#a archivo .csv (separado por comas) para poder trabajar con ello.

#Abrimos el archivo proporcionado por el laboratorio en modo lectura
with open('comas.csv') as File:
    reader = csv.reader(File, delimiter=',', quotechar=',',quoting=csv.QUOTE_MINIMAL)
    
    for row in reader: #loop que recorre todas las lineas del archivo

        time_start.append(row[3]) #añadimos al array time_start el valor asignado a la posicion 3 de la linea
        time_end.append(row[4]) #añadimos al array time_start el valor asignado a la posicion 4 de la linea
        
        if row[0] != 'ï»¿FILE NAME':
            video_name.append(row[0])
        
        if row[1] != 'RAT NUMBER':
            rat_number.append(row[1])
            

#para trabajar con los tiempos, como se ha hecho anteriormente, trabajaremos con una sola unidad (segundos)
seconds_start = [] #array donde almacenaremos los segundos en los que comienza el ataque
seconds_end = []   #array donde almacenaremos los segundos en los que finaliza el ataque

#2 loop para pasar el tiempo a la misma unidad.
for line in time_start: 
    
    if line != 'START':
        
        #el formato de hora en nuestro archivo viene marcado como PM o AM, lo que quiere decir que en el caso de ser
        #PM la hora reflejada en el excell sera 12 horas menos que en los videos, por lo que se las sumamos.
        if line[9:11] == 'PM':
            
            h_m = (int(line[0:2]) + 12) * 3600
            m_s = int(line[3:5]) * 60
            s = int(line[6:8])
            total = h_m + m_s + s

            seconds_start.append(str(total))
            
        if line[9:11] == 'AM':
            
            h_m = (int(line[0:2])) * 3600
            m_s = int(line[3:5]) * 60
            s = int(line[6:8])
            total = h_m + m_s + s
                
            seconds_start.append(str(total))
            
for line in time_end:
    
    if line != 'END':
        
        if line[9:11] == 'PM':
            
            h_m = (int(line[0:2]) + 12) * 3600
            m_s = int(line[3:5]) * 60
            s = int(line[6:8])
            total = h_m + m_s + s

                
            seconds_end.append(str(total))
            
        if line[9:11] == 'AM':
            
            h_m = (int(line[0:2])) * 3600
            m_s = int(line[3:5]) * 60
            s = int(line[6:8])
            total = h_m + m_s + s
                
            seconds_end.append(str(total))
            

for i in zip (video_name, rat_number, seconds_start, seconds_end): #loop que recorre todos los arrays a la vez
    
    #creamos los archivos .csv para los videos en los que disponemos de ataque
    name = str(i[0])
    a = name.find ('rat')
    b = i[1]
    archivo = name[:a] +'.Rat'+ b 
    
    header = ['Frame','Hours', 'Minutes', 'Seconds', 'State']
    
    #creamos el archivo y le escribimos la cabecera
    with open('RAT_VIDEOS_LABELLED/'+ archivo + '.csv', 'wt') as f:
        writer = csv.writer(f)
        writer.writerow(header)
    
    #abrimos el archivo creado sin estado con el mismo nombre en modo lectura
    with open ('RAT_VIDEOS_COMPLETED/'+ archivo +'.csv') as File:
        reader = csv.reader(File)
        
        for t in reader: #loop que recorre el archivo
            
            if len (t) == 4 and t[0]!= 'Frame':
                
                #Pasamos el tiempo a segundos
                h_m = int(t[1]) * 3600
                m_s = int(t[2]) * 60
                s = int(t[3])
                total = h_m + m_s + s
                
                if total >= int(i[2]) and total <= int (i[3]): #comparamos los segundos obtenidos con el tiempo en el
                                                                # que comienza el ataque y  finaliza
                    
                    #si los segundos estan comprendios entre el tiempo start y el tiempo end, el estado sera:
                    #epileptic attack (EA), de lo contrario sera repose (RE)
                    EA = ['EA']
                    StateA = t + EA
                    
                    with open('RAT_VIDEOS_LABELLED/'+ archivo + '.csv', 'a') as f:
                        writer = csv.writer(f)
                        writer.writerow(StateA)
                        
                else:
                    
                    
                    repose = ['RE']
                    StateB = t + repose 
                    
                    with open('RAT_VIDEOS_LABELLED/'+ archivo + '.csv', 'a') as f:
                        writer = csv.writer(f)
                        writer.writerow(StateB)
                        
    print (archivo + ' ' + 'completed')

1.2018-1-9.Rat9 completed
2.2017-11-10.Rat12 completed
3.2017-11-10.Rat12 completed
4.2017-11-10.Rat12 completed
5.2017-11-10.Rat12 completed
6.2017-11-10.Rat12 completed
7.2017-11-10.Rat12 completed
8.2017-11-10.Rat12 completed
9.2017-11-10.Rat12 completed
10.2017-11-10.Rat12 completed
11.2017-11-13.Rat12 completed
12.2017-12-13.Rat18 completed
13.2017-12-14.Rat18 completed
14.2017-12-14.Rat18 completed
15.2017-12-15.Rat18 completed
16.2018-1-19.Rat18 completed
17.2018-1-20.Rat18 completed
18.2017-12-22.Rat24 completed
19.2017-12-22.Rat24 completed
20.2017-12-22.Rat24 completed
21.2017-12-22.Rat24 completed
22.2017-12-22.Rat24 completed
23.2017-12-22.Rat24 completed
24.2017-12-22.Rat24 completed
25.2018-2-14.Rat25 completed
26.2018-2-14.Rat25 completed
27.2018-2-14.Rat25 completed
28.2018-2-15.Rat25 completed
29.2018-2-15.Rat25 completed
30.2018-2-15.Rat25 completed
31.2018-2-15.Rat25 completed
32.2018-2-15.Rat25 completed
33.2018-2-15.Rat25 completed
34.2018-2-15.Rat25 completed
35.2

In [35]:
##########################################################################################################################
# Una vez etiquetados todos los videos en los que tenemos ataque, procederemos a etiquetar los videos en los que no lo hay
##########################################################################################################################

#vamos a buscar los archivos no coincidentes con los archivos ya etiquetados.

#Rutas con los archivos 
extrapol = r'C:\Users\flum1\Desktop\Deep Learning\RAT_VIDEOS_COMPLETED'
etiquetados = r'C:\Users\flum1\Desktop\Deep Learning\RAT_VIDEOS_LABELLED'


tiempos_csv = os.listdir(extrapol) #array con los archivos de los tiempos extrapolados
estados_csv = os.listdir(etiquetados) #rray con los archivos que tienen estados


tiempos_sin_estados_csv = [] #array con los archivos no coincidentes


for t_csv in tiempos_csv: #loop que recorre el array de los tiempos extrapolados
    tiene_estados = False 
    
    for s_csv in estados_csv: #Para cada interacción recorremos un loop que recorre el array con los archivos etiquetados
        if s_csv == t_csv: #Si las variables coinciden, cambiamos tiene_estados a True
            tiene_estados = True
            
    if not tiene_estados: #Si no coinciden, añadimos a la lista el archivo no coincidente
        tiempos_sin_estados_csv.append( t_csv )

#Creamos los archivos .csv para cada archivo que falta sin etiquetar
header = ['Frame','Hours', 'Minutes', 'Seconds', 'State']

for line in tiempos_sin_estados_csv: #loop que recorre los archivos no coincidentes
    
    #creamos el archivo .csv
    with open('RAT_VIDEOS_LABELLED/'+ line, 'wt') as f:
        writer = csv.writer(f)
        writer.writerow(header)
    
    #abrimos el archivo creado sin estado con el mismo nombre en modo lectura
    with open ('RAT_VIDEOS_COMPLETED/'+ line) as File:
        reader = csv.reader(File)
        
        for t in reader: #loop que recorre el archivo
            
            if len (t) == 4 and t[0]!= 'Frame':
                repose = ['RE']
                StateB = t + repose

                with open('RAT_VIDEOS_LABELLED/'+ line, 'a') as f:
                    writer = csv.writer(f)
                    writer.writerow(StateB)
                
    print (line + ' ' + 'completed')

163.2018-2-13.Rat28.csv completed
163.2018-2-13.Rat34.csv completed
164.2018-2-13.Rat27.csv completed
164.2018-2-13.Rat28.csv completed
164.2018-2-13.Rat34.csv completed
165.2018-2-19.Rat27.csv completed
165.2018-2-19.Rat28.csv completed
165.2018-2-19.Rat34.csv completed
166.2018-2-19.Rat27.csv completed
166.2018-2-19.Rat28.csv completed
166.2018-2-19.Rat34.csv completed
167.2018-2-19.Rat27.csv completed
167.2018-2-19.Rat28.csv completed
167.2018-2-19.Rat34.csv completed
168.2018-2-19.Rat27.csv completed
168.2018-2-19.Rat28.csv completed
168.2018-2-19.Rat34.csv completed
169.2018-2-19.Rat27.csv completed
169.2018-2-19.Rat28.csv completed
169.2018-2-19.Rat34.csv completed
17.2018-1-20.Rat13.csv completed
170.2018-2-19.Rat27.csv completed
170.2018-2-19.Rat28.csv completed
170.2018-2-19.Rat34.csv completed
171.2018-2-19.Rat27.csv completed
171.2018-2-19.Rat28.csv completed
171.2018-2-19.Rat34.csv completed
172.2018-5-17.Rat54.csv completed
172.2018-5-17.Rat55.csv completed
172.2018-5-17.R

91.2018-2-14.Rat25.csv completed
91.2018-2-14.Rat26.csv completed
92.2018-2-14.Rat25.csv completed
92.2018-2-14.Rat26.csv completed
93.2018-2-6.Rat36.csv completed
94.2018-2-6.Rat36.csv completed
95.2018-2-6.Rat36.csv completed
96.2018-2-6.Rat36.csv completed
97.2018-2-6.Rat36.csv completed
98.2018-2-6.Rat36.csv completed
99.2018-2-6.Rat36.csv completed


In [66]:
##########################################################################################################################
# La siguiente función cambiara todos los videos a la misma resolución (512x512), ya que a la hora de recortarlos
# anteriormente, cada video tiene una resolucion distinta
# para machine learning necesitaremos una resolucion 2^n, por eso usaremos 512x512
##########################################################################################################################

def resize_video (directorio):
    
    video = cv2.VideoCapture(directorio)
    name = directorio[19:]
    
    video_resize = cv2.VideoWriter('RAT_VIDEOS_CROPPED_RESIZE/'+ name, cv2.VideoWriter_fourcc(*'XVID'),20.0,(512,512))
    
    while (video.isOpened()):
        ret, frame = video.read()

        if ret == True:
            
            
            frame_pill = Image.fromarray(frame) #abrimos el frame con Pillow
            x, y = frame_pill.size #x e y seran las dimensiones del frame
            size = 512 #resolucion que queremos
            
            #Redimensionamos la imagen, y los espacios sobrantes los rellenaremos de negro
            new_im = Image.new('RGBA', (size, size), (0, 0, 0, 0))
            new_im.paste(frame_pill, (int((size - x) / 2), int((size - y) / 2)))

            frame_np = np.asanyarray(new_im) #volvemos a pasar el frame a numpy
            
            frame_np_3 = frame_np[:,:,:3] #cogemos solo los 3 primeros canales
            
            video_resize.write(frame_np_3)
            
        if ret == False:
            break


    video_resize.release()

In [72]:
##########################################################################################################################
# Creamos una lista con todos los archivos que tenemos en la carpeta de los videos recortados, y ejecutamos la funcion 
# anterior a cada uno de ellos
##########################################################################################################################

#Le decimos la ruta 
ruta = r'C:\Users\flum1\Desktop\Deep Learning\RAT_VIDEOS_CROPPED'

#hacemos una lista de todos los archivos de la ruta
archivo = os.listdir(ruta)

#un loop para que ejecute el programa anterior a todos los elementos de la lista
for i in archivo:
    resize_video('RAT_VIDEOS_CROPPED/' + i)
    print (i + ' '+ 'completed')

1.2018-1-9.Rat8.mp4 completed
1.2018-1-9.Rat9.mp4 completed
10.2017-11-10.Rat11.mp4 completed
10.2017-11-10.Rat12.mp4 completed
100.2018-2-6.Rat34.mp4 completed
100.2018-2-6.Rat36.mp4 completed
101.2018-2-6.Rat34.mp4 completed
101.2018-2-6.Rat36.mp4 completed
102.2018-2-7.Rat34.mp4 completed
102.2018-2-7.Rat36.mp4 completed
103.2018-2-7.Rat34.mp4 completed
103.2018-2-7.Rat36.mp4 completed
104.2018-2-7.Rat34.mp4 completed
104.2018-2-7.Rat36.mp4 completed
105.2018-2-20.Rat34.mp4 completed
105.2018-2-20.Rat36.mp4 completed
106.2018-2-20.Rat34.mp4 completed
106.2018-2-20.Rat36.mp4 completed
107.2018-2-20.Rat34.mp4 completed
107.2018-2-20.Rat36.mp4 completed
108.2018-2-20.Rat34.mp4 completed
108.2018-2-20.Rat36.mp4 completed
109.2018-2-20.Rat34.mp4 completed
109.2018-2-20.Rat36.mp4 completed
11.2017-11-13.Rat11.mp4 completed
11.2017-11-13.Rat12.mp4 completed
110.2018-2-20.Rat34.mp4 completed
110.2018-2-20.Rat36.mp4 completed
111.2018-2-21.Rat34.mp4 completed
111.2018-2-21.Rat36.mp4 complete

183.2018-5-18.Rat54.mp4 completed
183.2018-5-18.Rat58.mp4 completed
184.2018-5-18.Rat52.mp4 completed
184.2018-5-18.Rat54.mp4 completed
184.2018-5-18.Rat58.mp4 completed
185.2018-5-18.Rat52.mp4 completed
185.2018-5-18.Rat54.mp4 completed
185.2018-5-18.Rat58.mp4 completed
186.2018-5-17.Rat52.mp4 completed
186.2018-5-17.Rat54.mp4 completed
186.2018-5-17.Rat55.mp4 completed
186.2018-5-17.Rat56.mp4 completed
186.2018-5-17.Rat58.mp4 completed
187.2018-5-17.Rat52.mp4 completed
187.2018-5-17.Rat54.mp4 completed
187.2018-5-17.Rat55.mp4 completed
187.2018-5-17.Rat56.mp4 completed
187.2018-5-17.Rat58.mp4 completed
188.2018-5-17.Rat52.mp4 completed
188.2018-5-17.Rat54.mp4 completed
188.2018-5-17.Rat55.mp4 completed
188.2018-5-17.Rat56.mp4 completed
188.2018-5-17.Rat58.mp4 completed
189.2018-5-17.Rat52.mp4 completed
189.2018-5-17.Rat54.mp4 completed
189.2018-5-17.Rat55.mp4 completed
189.2018-5-17.Rat56.mp4 completed
189.2018-5-17.Rat58.mp4 completed
19.2017-12-22.Rat24.mp4 completed
19.2017-12-22.

88.2018-2-14.Rat25.mp4 completed
88.2018-2-14.Rat26.mp4 completed
88.2018-2-14.Rat30.mp4 completed
89.2018-2-14.Rat25.mp4 completed
89.2018-2-14.Rat26.mp4 completed
89.2018-2-14.Rat30.mp4 completed
9.2017-11-10.Rat11.mp4 completed
9.2017-11-10.Rat12.mp4 completed
90.2018-2-14.Rat25.mp4 completed
90.2018-2-14.Rat26.mp4 completed
90.2018-2-14.Rat30.mp4 completed
91.2018-2-14.Rat25.mp4 completed
91.2018-2-14.Rat26.mp4 completed
91.2018-2-14.Rat30.mp4 completed
92.2018-2-14.Rat25.mp4 completed
92.2018-2-14.Rat26.mp4 completed
92.2018-2-14.Rat30.mp4 completed
93.2018-2-6.Rat34.mp4 completed
93.2018-2-6.Rat36.mp4 completed
94.2018-2-6.Rat34.mp4 completed
94.2018-2-6.Rat36.mp4 completed
95.2018-2-6.Rat34.mp4 completed
95.2018-2-6.Rat36.mp4 completed
96.2018-2-6.Rat34.mp4 completed
96.2018-2-6.Rat36.mp4 completed
97.2018-2-6.Rat34.mp4 completed
97.2018-2-6.Rat36.mp4 completed
98.2018-2-6.Rat34.mp4 completed
98.2018-2-6.Rat36.mp4 completed
99.2018-2-6.Rat34.mp4 completed
99.2018-2-6.Rat36.mp4 c

In [4]:
##########################################################################################################################
# Una vez terminado todo, procederemos a la creacion de los datasets.
##########################################################################################################################

def createDataset(video):

    it = 1 #Creamos un contador que nos indicara en que frame nos encontramos

    cap = cv2.VideoCapture(video) #archivo de video que vamos a leer
    b = video.find('.mp4') #Posicion del string en el que se encuentra '.mp4'
    name = video[54:b] #El nombre del archivo
    


    with h5py.File('/mnt/MD1200A/lconcha/videos/Datasets_fernando/' + name, 'w') as h5File: #Creamos en archivo .h5 

        while (cap.isOpened()): #leemos el video al cual vamos a crear su dataset

            ret, frame = cap.read()

            if ret == True:
                resize = cv2.resize(frame, (128,128), interpolation = cv2.INTER_AREA)
                Xset = h5File.create_dataset('X'+str(it), data= resize) # La variable X + numero de frame, se la asignaremos
                                                                       # a los frames 

                # A su vez abriremos el archivo .csv asociado al video y extraeremos el estado correspondiente de cada 
                # frame
                
                with open ('/mnt/MD1200A/lconcha/videos/RAT_VIDEOS_LABELLED_FERNANDO/'+ name + '.csv') as File:
                    reader = csv.reader(File)
                    
                    for t in reader: # loop que recorre el archivo .csv
                        if len (t) == 5 and t[0] == str(it): #Si la longitud de t es = 5 y el valor de la posicion t[0]
                                                             # (numero de frame) es igual al valor de it
                                
                            Yset = h5File.create_dataset('Y'+str(it), data = t[4]) # La variable Y + numero de frame, 
                                                                                   # se la asignaremos a los estados
                                                                        

                it += 1

            if ret == False:
                break

In [ ]:
#Le decimos la ruta 
ruta = r'/mnt/MD1200A/lconcha/videos/RAT_VIDEOS_CROPPED_RESIZE'

#hacemos una lista de todos los archivos de la ruta
archivo = os.listdir(ruta)

#un loop para que ejecute el programa anterior a todos los elementos de la lista
for i in archivo:
    createDataset('/mnt/MD1200A/lconcha/videos/RAT_VIDEOS_CROPPED_RESIZE/' + i)
    print (i + ' '+ 'completed')

1.2018-1-9.Rat8.mp4 completed
1.2018-1-9.Rat9.mp4 completed
10.2017-11-10.Rat11.mp4 completed
10.2017-11-10.Rat12.mp4 completed
11.2017-11-13.Rat11.mp4 completed
11.2017-11-13.Rat12.mp4 completed
12.2017-12-13.Rat13.mp4 completed
12.2017-12-13.Rat18.mp4 completed
13.2017-12-14.Rat13.mp4 completed
13.2017-12-14.Rat18.mp4 completed
14.2017-12-14.Rat13.mp4 completed
14.2017-12-14.Rat18.mp4 completed
15.2017-12-15.Rat13.mp4 completed
15.2017-12-15.Rat18.mp4 completed
16.2018-1-19.Rat13.mp4 completed
16.2018-1-19.Rat18.mp4 completed
17.2018-1-20.Rat13.mp4 completed
17.2018-1-20.Rat18.mp4 completed
18.2017-12-22.Rat24.mp4 completed
18.2017-12-22.Rat25.mp4 completed
18.2017-12-22.Rat28.mp4 completed
19.2017-12-22.Rat24.mp4 completed
19.2017-12-22.Rat25.mp4 completed
19.2017-12-22.Rat28.mp4 completed
2.2017-11-10.Rat11.mp4 completed
2.2017-11-10.Rat12.mp4 completed
20.2017-12-22.Rat24.mp4 completed
20.2017-12-22.Rat25.mp4 completed
20.2017-12-22.Rat28.mp4 completed
21.2017-12-22.Rat24.mp4 comp

134.2018-2-6.Rat36.mp4 completed
135.2018-2-6.Rat34.mp4 completed
135.2018-2-6.Rat36.mp4 completed
136.2018-2-6.Rat34.mp4 completed
136.2018-2-6.Rat36.mp4 completed
137.2018-2-6.Rat34.mp4 completed
137.2018-2-6.Rat36.mp4 completed
138.2018-2-6.Rat34.mp4 completed
138.2018-2-6.Rat36.mp4 completed
139.2018-2-6.Rat34.mp4 completed
139.2018-2-6.Rat36.mp4 completed
140.2018-2-6.Rat34.mp4 completed
140.2018-2-6.Rat36.mp4 completed
141.2018-2-6.Rat34.mp4 completed
141.2018-2-6.Rat36.mp4 completed
142.2018-2-6.Rat34.mp4 completed
142.2018-2-6.Rat36.mp4 completed
143.2018-2-7.Rat34.mp4 completed
143.2018-2-7.Rat36.mp4 completed
144.2018-2-7.Rat34.mp4 completed
144.2018-2-7.Rat36.mp4 completed
145.2018-2-7.Rat34.mp4 completed
145.2018-2-7.Rat36.mp4 completed
146.2018-2-7.Rat34.mp4 completed
146.2018-2-7.Rat36.mp4 completed
147.2018-2-7.Rat34.mp4 completed
147.2018-2-7.Rat36.mp4 completed
148.2018-2-7.Rat34.mp4 completed
148.2018-2-7.Rat36.mp4 completed
149.2018-2-7.Rat34.mp4 completed
149.2018-2

80.2018-2-14.Rat25.mp4 completed
80.2018-2-14.Rat26.mp4 completed
80.2018-2-14.Rat30.mp4 completed
81.2018-2-14.Rat25.mp4 completed
81.2018-2-14.Rat26.mp4 completed
81.2018-2-14.Rat30.mp4 completed
82.2018-2-14.Rat25.mp4 completed
82.2018-2-14.Rat26.mp4 completed
82.2018-2-14.Rat30.mp4 completed
83.2018-2-14.Rat25.mp4 completed
83.2018-2-14.Rat26.mp4 completed
83.2018-2-14.Rat30.mp4 completed
84.2018-2-14.Rat25.mp4 completed
84.2018-2-14.Rat26.mp4 completed
84.2018-2-14.Rat30.mp4 completed
85.2018-2-14.Rat25.mp4 completed
85.2018-2-14.Rat26.mp4 completed
85.2018-2-14.Rat30.mp4 completed
86.2018-2-14.Rat25.mp4 completed
86.2018-2-14.Rat26.mp4 completed
86.2018-2-14.Rat30.mp4 completed
87.2018-2-14.Rat25.mp4 completed
87.2018-2-14.Rat26.mp4 completed
87.2018-2-14.Rat30.mp4 completed
88.2018-2-14.Rat25.mp4 completed
88.2018-2-14.Rat26.mp4 completed
88.2018-2-14.Rat30.mp4 completed
89.2018-2-14.Rat25.mp4 completed
89.2018-2-14.Rat26.mp4 completed
89.2018-2-14.Rat30.mp4 completed
90.2018-2-

In [7]:
createDataset('/mnt/MD1200A/lconcha/videos/RAT_VIDEOS_CROPPED_RESIZE/1.2018-1-9.Rat8.mp4')